#### Setting up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/Shared drives/Adversarial NLP"

Mounted at /content/drive
/content/drive/Shared drives/Adversarial NLP


In [ ]:
!pip install textattack
!pip install transformers
!pip install evaluate
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 37.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.3/374.3 KB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 KB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.2 MB/s eta 0:00:00
  Preparing m

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import numpy as np
import math
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
import textattack
import transformers
from transformers import AutoTokenizer, RobertaModel, RobertaConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoModel, AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import PreTrainedTokenizer
import torch
import evaluate

from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_recipes import TextFoolerJin2019
from textattack.models.wrappers import ModelWrapper

import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaModel, RobertaTokenizer, RobertaForSequenceClassification
import torch.optim as optim
from torch.utils.data import DataLoader
from datasets import load_dataset

import random

from textattack.models.helpers import T5ForTextToText
from textattack.models.tokenizers import T5Tokenizer

from textattack.models.wrappers.pytorch_model_wrapper import PyTorchModelWrapper

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
train = load_dataset('ag_news', split='train')
test = load_dataset('ag_news', split='test')

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


In [ ]:
# raw model attack

# recipe: textfooler
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
# Load the dataset
dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")

Some weights of the model checkpoint at textattack/roberta-base-ag-news were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset ag_news, split test.


In [ ]:
# build the textfooler attack
attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_wrapper)

attack_args = textattack.AttackArgs(
    num_examples=10,
    log_to_csv="1_log_textfooler.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    csv_coloring_style="plain",
    disable_stdout=True
)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:38<00:00, 12.4MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp6fotmu7o.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path 1_log_textfooler.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:  50%|█████     | 5/10 [01:06<01:06, 13.40s/it]textattack: Saving checkpoint under "checkpoints/1679710560822.ta.chkpt" at 2023-03-25 02:16:00 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [01:43<00:00, 10.32s/it]textattack: Saving checkpoint under "checkpoints/1679710597013.ta.chkpt" at 2023-03-25 02:16:37 after 10 attacks.
[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [01:43<00:00, 10.32s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 16.07% |
| Average num. words per input: | 63.0   |
| Avg num queries:              | 469.7  |
+-------------------------------+--------+


In [ ]:
attack = textattack.attack_recipes.bert_attack_li_2020.BERTAttackLi2020.build(model_wrapper)

# Load the dataset
dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")

# run the attack
attack_args = textattack.AttackArgs(
    num_examples=10,
    log_to_csv="1_log_BERTattack.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    csv_coloring_style="plain",
    disable_stdout=True
)

transformation = WordSwapMaskedLM(max_length=128)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

#### Random masking

In [ ]:
torch.cuda.empty_cache()

# HuggingFaceModelWrapper
class MyWrapperRandomMask(PyTorchModelWrapper):
    """Loads a HuggingFace ``transformers`` model and tokenizer."""

    def __init__(self, model, tokenizer):
        assert isinstance(
            model, (transformers.PreTrainedModel, T5ForTextToText)
        ), f"`model` must be of type `transformers.PreTrainedModel`, but got type {type(model)}."
        assert isinstance(
            tokenizer,
            (
                transformers.PreTrainedTokenizer,
                transformers.PreTrainedTokenizerFast,
                T5Tokenizer,
            ),
        ), f"`tokenizer` must of type `transformers.PreTrainedTokenizer` or `transformers.PreTrainedTokenizerFast`, but got type {type(tokenizer)}."

        self.model = model
        self.tokenizer = tokenizer
        # self.tfidf_vocab = tfidf_vocab

    def __call__(self, text_input_list):
        """Passes inputs to HuggingFace models as keyword arguments.
        (Regular PyTorch ``nn.Module`` models typically take inputs as
        positional arguments.)
        """

        """
        Option 1 Make a custom preprocessing function for text_input_list
        """
#############################       
        percent_to_mask = 30
        for i in range(len(text_input_list)):
            words = text_input_list[i].split()
            num_words = len(words)
            num_to_mask = int(num_words * percent_to_mask / 100)
            indices_to_mask = random.sample(range(num_words), num_to_mask)
            text_input_list[i] = (" ".join(["<mask>" if i in indices_to_mask else word for i, word in enumerate(words)]))
#############################
        # Default max length is set to be int(1e30), so we force 512 to enable batching.
        max_length = (
            512
            if self.tokenizer.model_max_length == int(1e30)
            else self.tokenizer.model_max_length
        )
        inputs_dict = self.tokenizer(
            text_input_list,
            add_special_tokens=True,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            return_tensors="pt",
        )

       
        model_device = next(self.model.parameters()).device
        inputs_dict.to(model_device)

        with torch.no_grad():
            outputs = self.model(**inputs_dict)

        if isinstance(outputs[0], str):
            # HuggingFace sequence-to-sequence models return a list of
            # string predictions as output. In this case, return the full
            # list of outputs.
            return outputs
        else:
            # HuggingFace classification models return a tuple as output
            # where the first item in the tuple corresponds to the list of
            # scores for each input.
            return outputs.logits

    def get_grad(self, text_input):
        """Get gradient of loss with respect to input tokens.
        Args:
            text_input (str): input string
        Returns:
            Dict of ids, tokens, and gradient as numpy array.
        """
        if isinstance(self.model, textattack.models.helpers.T5ForTextToText):
            raise NotImplementedError(
                "`get_grads` for T5FotTextToText has not been implemented yet."
            )

        self.model.train()
        embedding_layer = self.model.get_input_embeddings()
        original_state = embedding_layer.weight.requires_grad
        embedding_layer.weight.requires_grad = True

        emb_grads = []

        def grad_hook(module, grad_in, grad_out):
            emb_grads.append(grad_out[0])

        emb_hook = embedding_layer.register_backward_hook(grad_hook)

        self.model.zero_grad()
        model_device = next(self.model.parameters()).device
        input_dict = self.tokenizer(
            [text_input],
            add_special_tokens=True,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
        )
        input_dict.to(model_device)
        predictions = self.model(**input_dict).logits

        try:
            labels = predictions.argmax(dim=1)
            loss = self.model(**input_dict, labels=labels)[0]
        except TypeError:
            raise TypeError(
                f"{type(self.model)} class does not take in `labels` to calculate loss. "
                "One cause for this might be if you instantiatedyour model using `transformer.AutoModel` "
                "(instead of `transformers.AutoModelForSequenceClassification`)."
            )

        loss.backward()

        # grad w.r.t to word embeddings
        grad = emb_grads[0][0].cpu().numpy()

        embedding_layer.weight.requires_grad = original_state
        emb_hook.remove()
        self.model.eval()

        output = {"ids": input_dict["input_ids"], "gradient": grad}

        return output

    def _tokenize(self, inputs):
        """Helper method that for `tokenize`
        Args:
            inputs (list[str]): list of input strings
        Returns:
            tokens (list[list[str]]): List of list of tokens as strings
        """
        return [
            self.tokenizer.convert_ids_to_tokens(
                self.tokenizer([x], truncation=True)["input_ids"][0]
            )
            for x in inputs
        ]

In [ ]:
# first generate adversarial examples using textattack
# recipe: textfooler
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")
model_wrapper = MyWrapperRandomMask(model, tokenizer)

# build the textfooler attack
attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_wrapper)

# Load the dataset
dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")
# dataset = load_dataset("ag_news")

Some weights of the model checkpoint at textattack/roberta-base-ag-news were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:09<00:00, 49.7MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp8bufqz6g.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embed

  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset ag_news, split test.


In [ ]:
# Attack 100 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(
    num_examples=200,
    log_to_csv="<masking>_words_30.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=True
)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path <masking>_words_30.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 1 / 1 / 5:   2%|▎         | 5/200 [00:54<35:40, 10.98s/it]textattack: Saving checkpoint under "checkpoints/1678853289022.ta.chkpt" at 2023-03-15 04:08:09 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 4 / 1 / 10:   5%|▌         | 10/200 [03:41<1:10:11, 22.17s/it]textattack: Saving checkpoint under "checkpoints/1678853455802.ta.chkpt" at 2023-03-15 04:10:55 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 6 / 1 / 15:   8%|▊         | 15/200 [04:52<1:00:05, 19.49s/it]textattack: Saving checkpoint under "checkpoints/1678853526511.ta.chkpt" at 2023-03-15 04:12:06 after 15 attacks.
[Succeeded / Failed / Skipped / Total] 8 / 6 / 2 / 16:   8%|▊         | 16/200 [04:52<56:02, 18.28s/it]  

...

[Succeeded / Failed / Skipped / Total] 107 / 62 / 16 / 185:  92%|█████████▎| 185/200 [34:29<02:47, 11.19s/it]textattack: Saving checkpoint under "checkpoints/1678855304012.ta.chkpt" at 2023-03-15 04:41:44 after 185 attacks.


[Succeeded / Failed / Skipped / Total] 110 / 63 / 17 / 190:  95%|█████████▌| 190/200 [35:25<01:51, 11.19s/it]textattack: Saving checkpoint under "checkpoints/1678855359607.ta.chkpt" at 2023-03-15 04:42:39 after 190 attacks.


[Succeeded / Failed / Skipped / Total] 110 / 68 / 17 / 195:  98%|█████████▊| 195/200 [37:21<00:57, 11.49s/it]textattack: Saving checkpoint under "checkpoints/1678855475220.ta.chkpt" at 2023-03-15 04:44:35 after 195 attacks.


[Succeeded / Failed / Skipped / Total] 111 / 71 / 18 / 200: 100%|██████████| 200/200 [38:15<00:00, 11.48s/it]textattack: Saving checkpoint under "checkpoints/1678855529192.ta.chkpt" at 2023-03-15 04:45:29 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 111 / 71 / 18 / 200: 100%|██████████| 200/200 [38:15<00:00, 11.48s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 111    |
| Number of failed attacks:     | 71     |
| Number of skipped attacks:    | 18     |
| Original accuracy:            | 91.0%  |
| Accuracy under attack:        | 35.5%  |
| Attack success rate:          | 60.99% |
| Average perturbed word %:     | 8.45%  |
| Average num. words per input: | 41.98  |
| Avg num queries:              | 381.13 |
+-------------------------------+--------+


#### Word replacing with tfidf (importance_dict)

In [ ]:
torch.cuda.empty_cache()

# HuggingFaceModelWrapper
class MyWrapperImportanceBasedMask(PyTorchModelWrapper):
    """Loads a HuggingFace ``transformers`` model and tokenizer."""

    def __init__(self, model, tokenizer, importance_dict):
        assert isinstance(
            model, (transformers.PreTrainedModel, T5ForTextToText)
        ), f"`model` must be of type `transformers.PreTrainedModel`, but got type {type(model)}."
        assert isinstance(
            tokenizer,
            (
                transformers.PreTrainedTokenizer,
                transformers.PreTrainedTokenizerFast,
                T5Tokenizer,
            ),
        ), f"`tokenizer` must of type `transformers.PreTrainedTokenizer` or `transformers.PreTrainedTokenizerFast`, but got type {type(tokenizer)}."

        self.model = model
        self.tokenizer = tokenizer
        self.importance_dict = importance_dict

    def __call__(self, text_input_list):
        """Passes inputs to HuggingFace models as keyword arguments.
        (Regular PyTorch ``nn.Module`` models typically take inputs as
        positional arguments.)
        """

        """
        Option 1 Put preprocessing function for text_input_list before tokenizing
        """
#############################       
        percent_to_mask = 30
        for i in range(len(text_input_list)):
            # truncate n% of words using tfidf
            words = text_input_list[i].split()

            # Sort the words in descending order of importance
            sorted_words = sorted(words, key=lambda w: self.importance_dict.get(w, 0), reverse=True)

            # Replace the n most important words with "<mask>"
            n_words_to_mask = int(len(sorted_words) * (int(percent_to_mask) / 100))
            masked_words = ["<mask>" if w in sorted_words[:n_words_to_mask] else w for w in words]

            # Join the remaining words back into a sentence
            remaining_sentence = " ".join(masked_words)

            text_input_list[i] = remaining_sentence
#############################

        # Default max length is set to be int(1e30), so we force 512 to enable batching.
        max_length = (
            512
            if self.tokenizer.model_max_length == int(1e30)
            else self.tokenizer.model_max_length
        )
        inputs_dict = self.tokenizer(
            text_input_list,
            add_special_tokens=True,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            return_tensors="pt",
        )

        model_device = next(self.model.parameters()).device
        inputs_dict.to(model_device)

        with torch.no_grad():
            outputs = self.model(**inputs_dict)

        if isinstance(outputs[0], str):
            # HuggingFace sequence-to-sequence models return a list of
            # string predictions as output. In this case, return the full
            # list of outputs.
            return outputs
        else:
            # HuggingFace classification models return a tuple as output
            # where the first item in the tuple corresponds to the list of
            # scores for each input.
            return outputs.logits

    def get_grad(self, text_input):
        """Get gradient of loss with respect to input tokens.
        Args:
            text_input (str): input string
        Returns:
            Dict of ids, tokens, and gradient as numpy array.
        """
        if isinstance(self.model, textattack.models.helpers.T5ForTextToText):
            raise NotImplementedError(
                "`get_grads` for T5FotTextToText has not been implemented yet."
            )

        self.model.train()
        embedding_layer = self.model.get_input_embeddings()
        original_state = embedding_layer.weight.requires_grad
        embedding_layer.weight.requires_grad = True

        emb_grads = []

        def grad_hook(module, grad_in, grad_out):
            emb_grads.append(grad_out[0])

        emb_hook = embedding_layer.register_backward_hook(grad_hook)

        self.model.zero_grad()
        model_device = next(self.model.parameters()).device
        input_dict = self.tokenizer(
            [text_input],
            add_special_tokens=True,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
        )
        input_dict.to(model_device)
        predictions = self.model(**input_dict).logits

        try:
            labels = predictions.argmax(dim=1)
            loss = self.model(**input_dict, labels=labels)[0]
        except TypeError:
            raise TypeError(
                f"{type(self.model)} class does not take in `labels` to calculate loss. "
                "One cause for this might be if you instantiatedyour model using `transformer.AutoModel` "
                "(instead of `transformers.AutoModelForSequenceClassification`)."
            )

        loss.backward()

        # grad w.r.t to word embeddings
        grad = emb_grads[0][0].cpu().numpy()

        embedding_layer.weight.requires_grad = original_state
        emb_hook.remove()
        self.model.eval()

        output = {"ids": input_dict["input_ids"], "gradient": grad}

        return output

    def _tokenize(self, inputs):
        """Helper method that for `tokenize`
        Args:
            inputs (list[str]): list of input strings
        Returns:
            tokens (list[list[str]]): List of list of tokens as strings
        """
        return [
            self.tokenizer.convert_ids_to_tokens(
                self.tokenizer([x], truncation=True)["input_ids"][0]
            )
            for x in inputs
        ]

In [ ]:
# IMPROVE THIS
# tfidf for words
# maybe implement removing stop words from the corpus and maybe implement lemmatization?
from collections import Counter
from nltk.stem import WordNetLemmatizer

def tfidf_vocab(sentences):
    # lemmatizer = WordNetLemmatizer()

    # Step 1: Calculate the term frequency (TF) for each word in each sentence
    tf_dict = {}
    sentence_count = len(sentences)
    for i, sentence in enumerate(sentences):
        words = sentence.split()
        word_count = len(words)
        word_freq = Counter(words)
        for word in word_freq:
            tf = word_freq[word] / word_count
            if word in tf_dict:
                if i in tf_dict[word]:
                    tf_dict[word][i] += tf
                else:
                    tf_dict[word][i] = tf
            else:
                tf_dict[word] = {i: tf}
    
    # Step 2: Calculate the inverse document frequency (IDF) for each word
    idf_dict = {}
    for word in tf_dict:
        idf_dict[word] = math.log(sentence_count / len(tf_dict[word]))
    
    # Step 3: Calculate the TF-IDF score for each word in each sentence
    tfidf_dict = {}
    for word in tf_dict:
        for sentence_index in tf_dict[word]:
            tf = tf_dict[word][sentence_index]
            idf = idf_dict[word]
            tfidf = tf * idf
            if sentence_index in tfidf_dict:
                tfidf_dict[sentence_index][word] = tfidf
            else:
                tfidf_dict[sentence_index] = {word: tfidf}
    
    # Step 4: Sum up TF-IDF scores for each unique word
    tfidf_corpus = {}
    for i in range(len(tfidf_dict)):
        for word in tfidf_dict[i]:
            if word in tfidf_corpus:
                tfidf_corpus[word] += tfidf_dict[i][word]
            else:
                tfidf_corpus[word] = tfidf_dict[i][word]
 
    return tfidf_corpus

In [ ]:
tfidf_corpus = tfidf_vocab(train['text'])

In [ ]:
# remove stop words from the important word corpus
# more like stop words and words that probably won't be perturbed but are important (?)
def remove_stop_words(word_importance_dict):
    # add more 
    stop_words = ["the", "a", "an", "and", "in", "on", "at", "to", "of", "are", "is", "am", "were"]
    # Create a new dictionary that excludes the stop words
    return {k: v for k, v in word_importance_dict.items() if k not in stop_words}

tfidf_corpus = remove_stop_words(tfidf_corpus)

In [ ]:
# first generate adversarial examples using textattack
# recipe: textfooler
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")
model_wrapper = MyWrapperImportanceBasedMask(model, tokenizer, tfidf_corpus)

# build the textfooler attack
attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_wrapper)

# Load the dataset
dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")

# Attack 100 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(
    num_examples=200,
    log_to_csv="tfidf_30_<mask>.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=True
)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Some weights of the model checkpoint at textattack/roberta-base-ag-news were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset ag_news, split test.
textattack: Logging to CSV at path tfidf_30_<mask>.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   2%|▎         | 5/200 [00:45<29:53,  9.20s/it]textattack: Saving checkpoint under "checkpoints/1678855854448.ta.chkpt" at 2023-03-15 04:50:54 after 5 attacks.


...

[Succeeded / Failed / Skipped / Total] 146 / 25 / 14 / 185:  92%|█████████▎| 185/200 [28:42<02:19,  9.31s/it]textattack: Saving checkpoint under "checkpoints/1678857531039.ta.chkpt" at 2023-03-15 05:18:51 after 185 attacks.


[Succeeded / Failed / Skipped / Total] 151 / 25 / 14 / 190:  95%|█████████▌| 190/200 [29:16<01:32,  9.24s/it]textattack: Saving checkpoint under "checkpoints/1678857564635.ta.chkpt" at 2023-03-15 05:19:24 after 190 attacks.


[Succeeded / Failed / Skipped / Total] 155 / 26 / 14 / 195:  98%|█████████▊| 195/200 [30:20<00:46,  9.34s/it]textattack: Saving checkpoint under "checkpoints/1678857629162.ta.chkpt" at 2023-03-15 05:20:29 after 195 attacks.


[Succeeded / Failed / Skipped / Total] 159 / 27 / 14 / 200: 100%|██████████| 200/200 [31:04<00:00,  9.32s/it]textattack: Saving checkpoint under "checkpoints/1678857672863.ta.chkpt" at 2023-03-15 05:21:12 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 159 / 27 / 14 / 200: 100%|██████████| 200/200 [31:04<00:00,  9.32s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 159    |
| Number of failed attacks:     | 27     |
| Number of skipped attacks:    | 14     |
| Original accuracy:            | 93.0%  |
| Accuracy under attack:        | 13.5%  |
| Attack success rate:          | 85.48% |
| Average perturbed word %:     | 19.6%  |
| Average num. words per input: | 41.98  |
| Avg num queries:              | 306.97 |
+-------------------------------+--------+


#### Random masking on vectorized tokens

In [ ]:
torch.cuda.empty_cache()

# HuggingFaceModelWrapper
class MyWrapperRandomTokenMask(PyTorchModelWrapper):
    """Loads a HuggingFace ``transformers`` model and tokenizer."""

    def __init__(self, model, tokenizer):
        assert isinstance(
            model, (transformers.PreTrainedModel, T5ForTextToText)
        ), f"`model` must be of type `transformers.PreTrainedModel`, but got type {type(model)}."
        assert isinstance(
            tokenizer,
            (
                transformers.PreTrainedTokenizer,
                transformers.PreTrainedTokenizerFast,
                T5Tokenizer,
            ),
        ), f"`tokenizer` must of type `transformers.PreTrainedTokenizer` or `transformers.PreTrainedTokenizerFast`, but got type {type(tokenizer)}."

        self.model = model
        self.tokenizer = tokenizer
        # self.tfidf_vocab = tfidf_vocab

    def __call__(self, text_input_list):
        """Passes inputs to HuggingFace models as keyword arguments.
        (Regular PyTorch ``nn.Module`` models typically take inputs as
        positional arguments.)
        """

        # Default max length is set to be int(1e30), so we force 512 to enable batching.
        max_length = (
            512
            if self.tokenizer.model_max_length == int(1e30)
            else self.tokenizer.model_max_length
        )
        inputs_dict = self.tokenizer(
            text_input_list,
            add_special_tokens=True,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            return_tensors="pt",
        )

        """
        Option 2 Make a function that masks vectorized tokens
        """
#####################
        # take in the inputs_dict and randomly attention_mask = 0
        # Mask random tokens        
        # don't touch the first and last tokens
        percent_zero=40
        for i in range(len(inputs_dict['attention_mask'])):
            length = torch.count_nonzero(inputs_dict['attention_mask'][i])-2
            num_zeros = int(length * percent_zero/100)
            indices = np.arange(length)
            random_indices = np.random.choice(np.arange(length), size=num_zeros, replace=False)
            for j in random_indices:
                inputs_dict['attention_mask'][i][j+1] = 0
        
######################

        model_device = next(self.model.parameters()).device
        inputs_dict.to(model_device)

        with torch.no_grad():
            outputs = self.model(**inputs_dict)

        if isinstance(outputs[0], str):
            # HuggingFace sequence-to-sequence models return a list of
            # string predictions as output. In this case, return the full
            # list of outputs.
            return outputs
        else:
            # HuggingFace classification models return a tuple as output
            # where the first item in the tuple corresponds to the list of
            # scores for each input.
            return outputs.logits

    def get_grad(self, text_input):
        """Get gradient of loss with respect to input tokens.
        Args:
            text_input (str): input string
        Returns:
            Dict of ids, tokens, and gradient as numpy array.
        """
        if isinstance(self.model, textattack.models.helpers.T5ForTextToText):
            raise NotImplementedError(
                "`get_grads` for T5FotTextToText has not been implemented yet."
            )

        self.model.train()
        embedding_layer = self.model.get_input_embeddings()
        original_state = embedding_layer.weight.requires_grad
        embedding_layer.weight.requires_grad = True

        emb_grads = []

        def grad_hook(module, grad_in, grad_out):
            emb_grads.append(grad_out[0])

        emb_hook = embedding_layer.register_backward_hook(grad_hook)

        self.model.zero_grad()
        model_device = next(self.model.parameters()).device
        input_dict = self.tokenizer(
            [text_input],
            add_special_tokens=True,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
        )
###
        percent_zero=40
        for i in range(len(inputs_dict['attention_mask'])):
            length = torch.count_nonzero(inputs_dict['attention_mask'][i])-2
            num_zeros = int(length * percent_zero/100)
            indices = np.arange(length)
            random_indices = np.random.choice(np.arange(length), size=num_zeros, replace=False)
            for j in random_indices:
                inputs_dict['attention_mask'][i][j+1] = 0
###
        input_dict.to(model_device)
        predictions = self.model(**input_dict).logits

        try:
            labels = predictions.argmax(dim=1)
            loss = self.model(**input_dict, labels=labels)[0]
        except TypeError:
            raise TypeError(
                f"{type(self.model)} class does not take in `labels` to calculate loss. "
                "One cause for this might be if you instantiatedyour model using `transformer.AutoModel` "
                "(instead of `transformers.AutoModelForSequenceClassification`)."
            )

        loss.backward()

        # grad w.r.t to word embeddings
        grad = emb_grads[0][0].cpu().numpy()

        embedding_layer.weight.requires_grad = original_state
        emb_hook.remove()
        self.model.eval()

        output = {"ids": input_dict["input_ids"], "gradient": grad}

        return output

    def _tokenize(self, inputs):
        """Helper method that for `tokenize`
        Args:
            inputs (list[str]): list of input strings
        Returns:
            tokens (list[list[str]]): List of list of tokens as strings
        """
        return [
            self.tokenizer.convert_ids_to_tokens(
                self.tokenizer([x], truncation=True)["input_ids"][0]
            )
            for x in inputs
        ]

In [ ]:
# first generate adversarial examples using textattack
# recipe: textfooler
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")
model_wrapper = MyWrapperRandomTokenMask(model, tokenizer)

# build the textfooler attack
attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_wrapper)

# Load the dataset
dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")

# Attack 100 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(
    num_examples=200,
    log_to_csv="random_masking_tokens_40.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=True
)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Some weights of the model checkpoint at textattack/roberta-base-ag-news were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset ag_news, split test.
textattack: Logging to CSV at path log.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   2%|▎         | 5/200 [00:38<25:03,  7.71s/it]textattack: Saving checkpoint under "checkpoints/1678218794412.ta.chkpt" at 2023-03-07 19:53:14 after 5 attacks.


...

[Succeeded / Failed / Skipped / Total] 136 / 38 / 11 / 185:  92%|█████████▎| 185/200 [29:05<02:21,  9.44s/it]textattack: Saving checkpoint under "checkpoints/1678220501716.ta.chkpt" at 2023-03-07 20:21:41 after 185 attacks.


[Succeeded / Failed / Skipped / Total] 140 / 38 / 12 / 190:  95%|█████████▌| 190/200 [29:59<01:34,  9.47s/it]textattack: Saving checkpoint under "checkpoints/1678220555486.ta.chkpt" at 2023-03-07 20:22:35 after 190 attacks.


[Succeeded / Failed / Skipped / Total] 143 / 40 / 12 / 195:  98%|█████████▊| 195/200 [31:22<00:48,  9.65s/it]textattack: Saving checkpoint under "checkpoints/1678220638182.ta.chkpt" at 2023-03-07 20:23:58 after 195 attacks.


[Succeeded / Failed / Skipped / Total] 148 / 40 / 12 / 200: 100%|██████████| 200/200 [31:48<00:00,  9.54s/it]textattack: Saving checkpoint under "checkpoints/1678220664388.ta.chkpt" at 2023-03-07 20:24:24 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 148 / 40 / 12 / 200: 100%|██████████| 200/200 [31:48<00:00,  9.54s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 148    |
| Number of failed attacks:     | 40     |
| Number of skipped attacks:    | 12     |
| Original accuracy:            | 94.0%  |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 78.72% |
| Average perturbed word %:     | 7.36%  |
| Average num. words per input: | 41.98  |
| Avg num queries:              | 292.99 |
+-------------------------------+--------+


#### Masking vectorized tokens with tfidf

In [ ]:
torch.cuda.empty_cache()

# HuggingFaceModelWrapper
class MyWrapperTfidfTokenMask(PyTorchModelWrapper):
    """Loads a HuggingFace ``transformers`` model and tokenizer."""

    def __init__(self, model, tokenizer, tfidf):
        assert isinstance(
            model, (transformers.PreTrainedModel, T5ForTextToText)
        ), f"`model` must be of type `transformers.PreTrainedModel`, but got type {type(model)}."
        assert isinstance(
            tokenizer,
            (
                transformers.PreTrainedTokenizer,
                transformers.PreTrainedTokenizerFast,
                T5Tokenizer,
            ),
        ), f"`tokenizer` must of type `transformers.PreTrainedTokenizer` or `transformers.PreTrainedTokenizerFast`, but got type {type(tokenizer)}."

        self.model = model
        self.tokenizer = tokenizer
        self.tfidf = tfidf
        # self.tfidf_vocab = tfidf_vocab

    def __call__(self, text_input_list):
        """Passes inputs to HuggingFace models as keyword arguments.
        (Regular PyTorch ``nn.Module`` models typically take inputs as
        positional arguments.)
        """

        # Default max length is set to be int(1e30), so we force 512 to enable batching.
        max_length = (
            512
            if self.tokenizer.model_max_length == int(1e30)
            else self.tokenizer.model_max_length
        )
        inputs_dict = self.tokenizer(
            text_input_list,
            add_special_tokens=True,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            return_tensors="pt",
        )

        """
        Option 2 Make a function that masks vectorized tokens
        """
#####################
        # take in the inputs_dict and attention_mask = 0
        # Mask tokens with the highest tfidf value      
        # don't touch the first and last tokens
        percent_zero=40
        for i in range(len(inputs_dict['input_ids'])):
            length = torch.count_nonzero(inputs_dict['attention_mask'][i])-2
            num_zeros = int(length * percent_zero/100)
            
            input_id = inputs_dict['input_ids'][i][1:(length+1)].tolist()

            sorted_list = sorted(input_id, key=lambda x: self.tfidf.get(x, 0), reverse=True)
            top_four = sorted_list[:num_zeros]
            indices = [input_id.index(x) for x in top_four]
            for j in indices:
                inputs_dict['attention_mask'][i][j+1] = 0
######################

        model_device = next(self.model.parameters()).device
        inputs_dict.to(model_device)

        with torch.no_grad():
            outputs = self.model(**inputs_dict)

        if isinstance(outputs[0], str):
            # HuggingFace sequence-to-sequence models return a list of
            # string predictions as output. In this case, return the full
            # list of outputs.
            return outputs
        else:
            # HuggingFace classification models return a tuple as output
            # where the first item in the tuple corresponds to the list of
            # scores for each input.
            return outputs.logits

    def get_grad(self, text_input):
        """Get gradient of loss with respect to input tokens.
        Args:
            text_input (str): input string
        Returns:
            Dict of ids, tokens, and gradient as numpy array.
        """
        if isinstance(self.model, textattack.models.helpers.T5ForTextToText):
            raise NotImplementedError(
                "`get_grads` for T5FotTextToText has not been implemented yet."
            )

        self.model.train()
        embedding_layer = self.model.get_input_embeddings()
        original_state = embedding_layer.weight.requires_grad
        embedding_layer.weight.requires_grad = True

        emb_grads = []

        def grad_hook(module, grad_in, grad_out):
            emb_grads.append(grad_out[0])

        emb_hook = embedding_layer.register_backward_hook(grad_hook)

        self.model.zero_grad()
        model_device = next(self.model.parameters()).device
        input_dict = self.tokenizer(
            [text_input],
            add_special_tokens=True,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
        )
        
        percent_zero=40
        for i in range(len(inputs_dict['input_ids'])):
            length = torch.count_nonzero(inputs_dict['attention_mask'][i])-2
            num_zeros = int(length * percent_zero/100)
            
            input_id = inputs_dict['input_ids'][i][1:(length+1)].tolist()

            sorted_list = sorted(input_id, key=lambda x: self.tfidf.get(x, 0), reverse=True)
            top_four = sorted_list[:num_zeros]
            indices = [input_id.index(x) for x in top_four]
            for j in indices:
                inputs_dict['attention_mask'][i][j+1] = 0

        input_dict.to(model_device)
        predictions = self.model(**input_dict).logits

        try:
            labels = predictions.argmax(dim=1)
            loss = self.model(**input_dict, labels=labels)[0]
        except TypeError:
            raise TypeError(
                f"{type(self.model)} class does not take in `labels` to calculate loss. "
                "One cause for this might be if you instantiatedyour model using `transformer.AutoModel` "
                "(instead of `transformers.AutoModelForSequenceClassification`)."
            )

        loss.backward()

        # grad w.r.t to word embeddings
        grad = emb_grads[0][0].cpu().numpy()

        embedding_layer.weight.requires_grad = original_state
        emb_hook.remove()
        self.model.eval()

        output = {"ids": input_dict["input_ids"], "gradient": grad}

        return output

    def _tokenize(self, inputs):
        """Helper method that for `tokenize`
        Args:
            inputs (list[str]): list of input strings
        Returns:
            tokens (list[list[str]]): List of list of tokens as strings
        """
        return [
            self.tokenizer.convert_ids_to_tokens(
                self.tokenizer([x], truncation=True)["input_ids"][0]
            )
            for x in inputs
        ]

In [ ]:
# function for making tfidf corpus for vectorized tokens
# input is a list of list of vectorized tokens (input_ids)
def tfidf_vocab(input):
    # Step 1: Calculate the term frequency (TF) for each number in each list
    tf_dict = {}
    list_count = len(input)
    for i, sub_input in enumerate(input):
        for num in sub_input:
            # Increment the TF count for this number in this list
            if num in tf_dict:
                if i in tf_dict[num]:
                    tf_dict[num][i] += 1
                else:
                    tf_dict[num][i] = 1
            else:
                tf_dict[num] = {i: 1}
    
    # Step 2: Calculate the inverse document frequency (IDF) for each number
    idf_dict = {}
    for num in tf_dict:
        idf_dict[num] = math.log(list_count / len(tf_dict[num]))
    
    # Step 3: Calculate the TF-IDF score for each number in each list
    tfidf_dict = {}
    for num in tf_dict:
        for list_index in tf_dict[num]:
            tf = tf_dict[num][list_index]
            idf = idf_dict[num]
            tfidf = tf * idf
            if list_index in tfidf_dict:
                tfidf_dict[list_index][num] = tfidf
            else:
                tfidf_dict[list_index] = {num: tfidf}
    
    # Step 4: sum up tfidf values for each unique tokens
    tfidf_corpus = {}
    for i in range(len(tfidf_dict)):
        for j in tfidf_dict[i]:
            if j in tfidf_corpus:
                tfidf_corpus[j] += tfidf_dict[i][j]
            else:
                tfidf_corpus[j] = tfidf_dict[i][j]
 
    return tfidf_corpus

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")
train_tokens = tokenizer(train['text'])
tfidf = tfidf_vocab(train_tokens['input_ids'])

In [ ]:
# first generate adversarial examples using textattack
# recipe: textfooler
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")
model_wrapper = MyWrapperTfidfTokenMask(model, tokenizer, tfidf)

# build the textfooler attack
attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_wrapper)

# Load the dataset
dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")

# Attack 100 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(
    num_examples=200,
    log_to_csv="tfidf_masking_tokens_40.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=True
)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Some weights of the model checkpoint at textattack/roberta-base-ag-news were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:38<00:00, 12.4MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpgo0bql2b.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embed

  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset ag_news, split test.
textattack: Logging to CSV at path tfidf_masking_tokens_40.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   2%|▎         | 5/200 [01:38<1:04:20, 19.80s/it]textattack: Saving checkpoint under "checkpoints/1678486420326.ta.chkpt" at 2023-03-10 22:13:40 after 5 attacks.


...

[Succeeded / Failed / Skipped / Total] 150 / 16 / 19 / 185:  92%|█████████▎| 185/200 [27:59<02:16,  9.08s/it]textattack: Saving checkpoint under "checkpoints/1678488000940.ta.chkpt" at 2023-03-10 22:40:00 after 185 attacks.


[Succeeded / Failed / Skipped / Total] 155 / 16 / 19 / 190:  95%|█████████▌| 190/200 [28:37<01:30,  9.04s/it]textattack: Saving checkpoint under "checkpoints/1678488038938.ta.chkpt" at 2023-03-10 22:40:38 after 190 attacks.


[Succeeded / Failed / Skipped / Total] 159 / 17 / 19 / 195:  98%|█████████▊| 195/200 [29:50<00:45,  9.18s/it]textattack: Saving checkpoint under "checkpoints/1678488111784.ta.chkpt" at 2023-03-10 22:41:51 after 195 attacks.


[Succeeded / Failed / Skipped / Total] 162 / 18 / 20 / 200: 100%|██████████| 200/200 [30:35<00:00,  9.18s/it]textattack: Saving checkpoint under "checkpoints/1678488156791.ta.chkpt" at 2023-03-10 22:42:36 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 162 / 18 / 20 / 200: 100%|██████████| 200/200 [30:35<00:00,  9.18s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 162    |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 20     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 9.0%   |
| Attack success rate:          | 90.0%  |
| Average perturbed word %:     | 21.99% |
| Average num. words per input: | 41.98  |
| Avg num queries:              | 314.37 |
+-------------------------------+--------+




---


#### testing random code

In [ ]:
train['text'][:2]

["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.']

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")

In [ ]:
text_input_list = train['text'][:2]

In [ ]:
percent_to_mask = 30
for i in range(len(text_input_list)):
    words = text_input_list[i].split()
    num_words = len(words)
    num_to_mask = int(num_words * percent_to_mask / 100)
    indices_to_mask = random.sample(range(num_words), num_to_mask)
    text_input_list[i] = (" ".join(["<mask>" if i in indices_to_mask else word for i, word in enumerate(words)]))

In [ ]:
text_input_list

["<mask> St. <mask> <mask> Back Into the Black (Reuters) <mask> <mask> Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are <mask> green again.",
 '<mask> Looks Toward Commercial Aerospace (Reuters) Reuters - Private <mask> <mask> <mask> Group,\\which has a reputation for <mask> well-timed <mask> occasionally\\controversial plays in <mask> <mask> industry, has quietly placed\\its bets on <mask> <mask> of the market.']

In [ ]:
inputs_dict = tokenizer(
    train['text'][:2],
    add_special_tokens=True,
    padding="max_length",
    max_length=512,
    truncation=True,
    return_tensors="pt",
)

        # take in the inputs_dict and randomly attention_mask = 0
        # Mask random tokens        
        # don't touch the first and last tokens
percent_zero=40
for i in range(len(inputs_dict['attention_mask'])):
    length = torch.count_nonzero(inputs_dict['attention_mask'][i])-2
    num_zeros = int(length * percent_zero/100)
    indices = np.arange(length)
    random_indices = np.random.choice(np.arange(length), size=num_zeros, replace=False)
    for j in random_indices:
        inputs_dict['attention_mask'][i][j+1] = 0
        


In [ ]:
inputs_dict

{'input_ids': tensor([[    0, 28216,   312,  ...,     1,     1,     1],
        [    0,  9518,   352,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 0, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [ ]:
inputs_dict

{'input_ids': tensor([[    0, 28216,   312,  ...,     1,     1,     1],
        [    0,  9518,   352,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 0, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

t — term (word)

d — document (set of words)

N — count of corpus

corpus — the total document set

tf-idf(t,d) = tf(t,d)*idf(t)

**tf-idf(t,d) = tf(t,d) * log(N/(df(t)+1))**

tf(t,d) = count of t in d / number of words in d

df(t) = occurrence of t in N documents

After getting the tf-idf values for each unique tokens in each 'document', sum the frequency values from each document for each unique vectorized tokens

In [ ]:
inputs_dict = tokenizer(
            train['text'][:1000]
        )

tfidf_vocab = {

  token : tfidf_value

}


In [ ]:
inputs_dict['input_ids'][:2]

In [ ]:
# function for making tfidf corpus for vectorized tokens
# input is a list of list of vectorized tokens (input_ids)
def tfidf_vocab(input):
    # Step 1: Calculate the term frequency (TF) for each number in each list
    tf_dict = {}
    list_count = len(input)
    for i, sub_input in enumerate(input):
        for num in sub_input:
            # Increment the TF count for this number in this list
            if num in tf_dict:
                if i in tf_dict[num]:
                    tf_dict[num][i] += 1
                else:
                    tf_dict[num][i] = 1
            else:
                tf_dict[num] = {i: 1}
    
    # Step 2: Calculate the inverse document frequency (IDF) for each number
    idf_dict = {}
    for num in tf_dict:
        idf_dict[num] = math.log(list_count / len(tf_dict[num]))
    
    # Step 3: Calculate the TF-IDF score for each number in each list
    tfidf_dict = {}
    for num in tf_dict:
        for list_index in tf_dict[num]:
            tf = tf_dict[num][list_index]
            idf = idf_dict[num]
            tfidf = tf * idf
            if list_index in tfidf_dict:
                tfidf_dict[list_index][num] = tfidf
            else:
                tfidf_dict[list_index] = {num: tfidf}
    
    # Step 4: sum up tfidf values for each unique tokens
    tfidf_corpus = {}
    for i in range(len(tfidf_dict)):
        for j in tfidf_dict[i]:
            if j in tfidf_corpus:
                tfidf_corpus[j] += tfidf_dict[i][j]
            else:
                tfidf_corpus[j] = tfidf_dict[i][j]
 
    return tfidf_corpus

In [ ]:
tfidf = tfidf_vocab(inputs_dict['input_ids'])

In [ ]:
inputs_dict = tokenizer(
    test['text'][:2],
    add_special_tokens=True,
    padding="max_length",
    max_length=128,
    truncation=True,
    return_tensors="pt",
)

In [ ]:
inputs_dict

{'input_ids': tensor([[    0,   597, 12541,    13,   255,   234,  4931,    71,  1431,  1890,
          2485,  4561,  1138,    23,  6980,  1437,  1437,   188,  1250,   224,
            51,    32,   128,  7779, 19051,   108,    71,  1431,    19, 35876,
          4095,   933,  1853, 18059,   922,     4,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,  

In [ ]:
percent_zero=40
for i in range(len(inputs_dict['input_ids'])):
    length = torch.count_nonzero(inputs_dict['attention_mask'][i])-2
    num_zeros = int(length * percent_zero/100)
    
    input_id = inputs_dict['input_ids'][i][1:(length+1)].tolist()

    sorted_list = sorted(input_id, key=lambda x: tfidf.get(x, 0), reverse=True)
    top_four = sorted_list[:num_zeros]
    indices = [input_id.index(x) for x in top_four]
    for j in indices:
        inputs_dict['attention_mask'][i][j+1] = 0


In [ ]:
inputs_dict

{'input_ids': tensor([[    0,   597, 12541,    13,   255,   234,  4931,    71,  1431,  1890,
          2485,  4561,  1138,    23,  6980,  1437,  1437,   188,  1250,   224,
            51,    32,   128,  7779, 19051,   108,    71,  1431,    19, 35876,
          4095,   933,  1853, 18059,   922,     4,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,  

In [ ]:
# tfidf for words
from collections import Counter

def tfidf_vocab(sentences):
    # Step 1: Calculate the term frequency (TF) for each word in each sentence
    tf_dict = {}
    sentence_count = len(sentences)
    for i, sentence in enumerate(sentences):
        words = sentence.split()
        word_count = len(words)
        word_freq = Counter(words)
        for word in word_freq:
            tf = word_freq[word] / word_count
            if word in tf_dict:
                if i in tf_dict[word]:
                    tf_dict[word][i] += tf
                else:
                    tf_dict[word][i] = tf
            else:
                tf_dict[word] = {i: tf}
    
    # Step 2: Calculate the inverse document frequency (IDF) for each word
    idf_dict = {}
    for word in tf_dict:
        idf_dict[word] = math.log(sentence_count / len(tf_dict[word]))
    
    # Step 3: Calculate the TF-IDF score for each word in each sentence
    tfidf_dict = {}
    for word in tf_dict:
        for sentence_index in tf_dict[word]:
            tf = tf_dict[word][sentence_index]
            idf = idf_dict[word]
            tfidf = tf * idf
            if sentence_index in tfidf_dict:
                tfidf_dict[sentence_index][word] = tfidf
            else:
                tfidf_dict[sentence_index] = {word: tfidf}
    
    # Step 4: Sum up TF-IDF scores for each unique word
    tfidf_corpus = {}
    for i in range(len(tfidf_dict)):
        for word in tfidf_dict[i]:
            if word in tfidf_corpus:
                tfidf_corpus[word] += tfidf_dict[i][word]
            else:
                tfidf_corpus[word] = tfidf_dict[i][word]
 
    return tfidf_corpus

In [ ]:
tfidf_vocab(train['text'][:6])

{'Wall': 0.17064375897410047,
 'St.': 0.08532187948705024,
 'Bears': 0.08532187948705024,
 'Claw': 0.08532187948705024,
 'Back': 0.08532187948705024,
 'Into': 0.08532187948705024,
 'the': 0.0,
 'Black': 0.08532187948705024,
 '(Reuters)': 0.02867337766621717,
 'Reuters': 0.02867337766621717,
 '-': 0.0,
 'Short-sellers,': 0.08532187948705024,
 "Street's": 0.08532187948705024,
 'dwindling\\band': 0.08532187948705024,
 'of': 0.07151518529586737,
 'ultra-cynics,': 0.08532187948705024,
 'are': 0.08283187890751621,
 'seeing': 0.08532187948705024,
 'green': 0.08532187948705024,
 'again.': 0.08532187948705024,
 'Carlyle': 0.09954219273489194,
 'Looks': 0.04977109636744597,
 'Toward': 0.04977109636744597,
 'Commercial': 0.04977109636744597,
 'Aerospace': 0.04977109636744597,
 'Private': 0.04977109636744597,
 'investment': 0.04977109636744597,
 'firm': 0.04977109636744597,
 'Group,\\which': 0.04977109636744597,
 'has': 0.09954219273489194,
 'a': 0.044154490198597324,
 'reputation': 0.049771096367



---



### ignore below

In [ ]:
import random
# Randomly mask n% of tokens (set attention_mask to 0)
class Random_mask(nn.Module):
    def __init__(self, percent_to_mask):
        super(Random_mask, self).__init__()
        self.percent_to_mask = percent_to_mask
        self.model = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news")

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):

        # Mask random tokens        
        # don't touch the first and last tokens
        num_zeros = int((self.percent_to_mask/100) * (len(attention_mask)-2))
        zero_indices = random.sample(range(len(attention_mask-2)), num_zeros)
        for i in zero_indices:
            attention_mask[i+1] = 0

        # return model(input_ids, attention_mask)
        return np.argmax(self.model(input_ids, attention_mask).logits.detach().numpy())

    def get_labels():
        return [0,1,2,3]


In [ ]:
from textattack.models.wrappers import ModelWrapper

class Model_wrapper(ModelWrapper):
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
    
    def __call__(self, text_input_list):
        results = []
        for sentence in text_input_list:
            results.append(self.model(**self.tokenizer(sentence, return_tensors="pt")))
        return results
    
    def get_labels(self):
        return self.model.get_labels()

In [ ]:
model_random = Random_mask(25)

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")
model_wrapper = Model_wrapper(tokenizer, model_random)

# build the textfooler attack
attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_wrapper)

In [ ]:
# Load the dataset
dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")

# Attack 100 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(
    num_examples=100,
    log_to_csv="log.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=True
)
# run the attack
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

### Method 1 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news", )
model = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news")

In [ ]:
class Method_1(nn.Module):
    def __init__(self, tfidf_vocab, percent_to_mask):
        super(Method_1, self).__init__()
        self.tfidf_vocab = tfidf_vocab
        self.percent_to_mask = percent_to_mask
        self.tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")
        self.model = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news")

    def forward(self, sentence):
    #     self,
    #     input_ids=None,
    #     attention_mask=None,
    #     token_type_ids=None,
    #     position_ids=None,
    #     head_mask=None,
    #     inputs_embeds=None,
    #     labels=None,
    #     output_attentions=None,
    #     output_hidden_states=None,
    #     return_dict=None,
    # ):
        # truncate n% of words using tfidf
        words = sentence.split()
        word_tfidf = [self.tfidf_vocab.get(word, 0) for word in words]
          
        # Sort the words by their TF-IDF value
        sorted_words = sorted(zip(words, word_tfidf),
                                key=lambda x: x[1], reverse=True)
          
        # Mask the specified number of words
        # note: I guess I can't use topk here since it's not tensors
        masked_sentence = sentence
        n_words_to_mask = int(len(sorted_words)*(int(self.percent_to_mask)/100))
        for word, word_tfidf in sorted_words[:n_words_to_mask]:
            masked_sentence = masked_sentence.replace(word, "") 
            # note: this replaces all the same words
            # so it could remove more words than needed
        sentence = masked_sentence
        return model(**tokenizer(sentence, return_tensors="pt"))


In [ ]:
# make the TF-IDF vocab/corpus
tfidf = TfidfVectorizer()
# Fit the TfidfVectorizer to the training data and transform the training data
tfidf.fit_transform(train['text'])
# Calculate the TF-IDF value for each word in the vocabulary
tfidf_vocab = tfidf.vocabulary_

In [ ]:
model_1 = Method_1(tfidf_vocab, 25)

Some weights of the model checkpoint at textattack/roberta-base-ag-news were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Use Textattack to attack the model

In [ ]:
# first generate adversarial examples using textattack
# recipe: textfooler
# model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news")
# tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")
# model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

# build the textfooler attack
attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_1)

# Load the dataset
dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:38<00:00, 12.5MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp2w68edjw.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset ag_news, split test.


In [ ]:
# Load the dataset
dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")

# Attack 100 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(
    num_examples=100,
    log_to_csv="log.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=True
)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Attempt to rewrite the logic in the tokenizer class so it's compatible with TextAttack

In [ ]:
class Method_1_tokenizer(PreTrainedTokenizer):
    def __init__(self, tfidf_vocab, **kwargs):
        super().__init__(**kwargs)
        self.tfidf_vocab = tfidf_vocab
        self.tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")

    def __call__(self, text):
        # add your custom logic here
        percent_to_mask = 25
        words = text.split()
        word_tfidf = [self.tfidf_vocab.get(word, 0) for word in words]
              
        # Sort the words by their TF-IDF value
        sorted_words = sorted(zip(words, word_tfidf),
                                key=lambda x: x[1], reverse=True)
              
        # Mask the specified number of words
        # note: I guess I can't use topk here since it's not tensors
        masked_text = text
        n_words_to_mask = int(len(sorted_words)*(int(percent_to_mask)/100))
        for word, word_tfidf in sorted_words[:n_words_to_mask]:
            masked_text = masked_text.replace(word, "") 
            # note: this replaces all the same words
            # so it could remove more words than needed
        return self.tokenizer(masked_text)

# def method_1_tokenizer(texts):
#     percent_to_mask = 25
#     masked_sentences = []
#     for sentence in texts:
#         words = sentence.split()
#         word_tfidf = [tfidf_vocab.get(word, 0) for word in words]
              
#         # Sort the words by their TF-IDF value
#         sorted_words = sorted(zip(words, word_tfidf),
#                                 key=lambda x: x[1], reverse=True)
              
#         # Mask the specified number of words
#         # note: I guess I can't use topk here since it's not tensors
#         masked_sentence = sentence
#         n_words_to_mask = int(len(sorted_words)*(int(percent_to_mask)/100))
#         for word, word_tfidf in sorted_words[:n_words_to_mask]:
#             masked_sentence = masked_sentence.replace(word, "") 
#             # note: this replaces all the same words
#             # so it could remove more words than needed

#     masked_sentences.append(masked_sentence)
    
#     tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news", )

#     return tokenizer(masked_sentences)

In [ ]:
# first generate adversarial examples using textattack
# recipe: textfooler
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news")
tokenizer = Method_1_tokenizer(tfidf_vocab)
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

# build the textfooler attack
attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_wrapper)

# Load the dataset
dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")

Some weights of the model checkpoint at textattack/roberta-base-ag-news were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset ag_news, split test.


In [ ]:
# Attack 100 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(
    num_examples=100,
    log_to_csv="log.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=True
)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path log.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


  0%|          | 0/100 [00:00<?, ?it/s]

TypeError: ignored



---



#### Method 1 - masking vectorized tokens (using pretrained tokenizer) with tfidf vocab and learnable weight

For huggingface model, output should be:


```
return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
```



In [ ]:
model(**tokenizer(test['text'][1], return_tensors="pt"))

SequenceClassifierOutput(loss=None, logits=tensor([[-0.4297, -3.1058, -2.1420,  5.8838]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
# model(input_ids (vectorized tokens), attention_mask)
str(np.argmax(model(**tokenizer(test['text'][1], return_tensors="pt")).logits.detach().numpy()))

'3'

array([[-0.4297115, -3.1057916, -2.1420312,  5.883754 ]], dtype=float32)

In [ ]:
tokenizer(test['text'][1], return_tensors="pt")

{'input_ids': tensor([[    0,   133,  8989,    16,   374,    35,  4665,  8075,  2711, 41385,
         22217, 10566,    13,  3861,  5374, 15801,    36,  4186, 15949,     4,
           175,    43, 44909,     4,   175,   111, 22331,  2191,  6390,     6,
           896,   480,    83,   200, 37457,  8687,     9,  7726, 32596,  5468,
            13,     5,  1437,   849,  3367,   131,   698,   153, 16866,  1512,
          1577,  9421,     6,    10,  3096,    13, 37457, 25943,  7223,  6140,
          2849, 28027,  8632,   980,  2524,     6,    34,  4142,   585,     5,
            78, 37457, 37757,  1248,    13,    63, 30107,  7726,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# Get the training and test datasets
train_data = dataset['train']['text']
test_data = dataset['test']['text']

    # Initialize a TfidfVectorizer
tfidf = TfidfVectorizer()

    # Fit the TfidfVectorizer to the training data and transform the training data
train_tfidf = tfidf.fit_transform(train_data)

    # Calculate the TF-IDF value for each word in the vocabulary
vocab_tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
len(vocab_tfidf)

65006

In [ ]:
doc = tokenizer(dataset['train']['text'][:5])['input_ids'] # use this as "tokenized docs"

In [ ]:
doc[4]

[0,
 32397,
 850,
 27283,
 7,
 70,
 12,
 958,
 638,
 6,
 12681,
 92,
 24213,
 7,
 382,
 866,
 36,
 11528,
 43,
 5040,
 111,
 255,
 4352,
 4384,
 232,
 681,
 850,
 6,
 27339,
 1527,
 2189,
 8,
 7031,
 8173,
 28811,
 6,
 1455,
 10,
 92,
 776,
 24213,
 6254,
 130,
 377,
 137,
 5,
 382,
 1939,
 1727,
 4,
 2]

In [ ]:
tf = doc[1]

In [ ]:
import math

def build_tfidf_vocab(documents):
    # Create a set of unique terms in all documents
    terms = set()
    for doc in documents:
        for term in doc:
            terms.add(term.lower())
    
    # Calculate document frequency for each term
    doc_freq = {term: 0 for term in terms}
    for doc in documents:
        for term in set(doc):
            doc_freq[term.lower()] += 1
    
    # Build the TF-IDF vocabulary
    vocab = {}
    num_docs = len(documents)
    for term in terms:
        tfidf_scores = []
        for doc in documents:
            tf = doc.count(term) / len(doc)
            idf = math.log(num_docs / doc_freq[term])
            tfidf_scores.append(tf * idf)
        vocab[term] = max(tfidf_scores)
        
    return vocab

In [ ]:
build_tfidf_vocab(doc)

AttributeError: ignored

In [ ]:
# use count vectorizer and use tfidftransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

CountVectorizer(tokenizer(dataset['train']['text'][0])["input_ids"])

TypeError: ignored

In [ ]:
from collections import Counter
import math
from transformers import AutoTokenizer

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('textattack/roberta-base-ag-news')

# Example list of texts
texts = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
]

# Tokenize the texts
tokenized_texts = [tokenizer.tokenize(text) for text in texts]

# Get the frequency of each token
token_counts = Counter([token for text_tokens in tokenized_texts for token in text_tokens])

# Calculate the inverse document frequency (IDF) of each token
num_documents = len(texts)
idf_values = {}
for token in token_counts.keys():
    doc_occurrences = sum([1 for text_tokens in tokenized_texts if token in text_tokens])
    idf_values[token] = math.log(num_documents / doc_occurrences)

# Get the TF-IDF value for each token in each text
tfidf_values = []
for i, text_tokens in enumerate(tokenized_texts):
    tfidf_doc = {}
    for token in set(text_tokens):
        tf_value = text_tokens.count(token) / len(text_tokens)
        tfidf_value = tf_value * idf_values[token]
        tfidf_doc[token] = tfidf_value
    tfidf_values.append(tfidf_doc)

# Print the TF-IDF values for each text
for i, tfidf_doc in enumerate(tfidf_values):
    print(f"Text {i+1}:")
    for token, value in tfidf_doc.items():
        print(f"\t{token}: {value:.4f}")

Text 1:
	Ġthe: 0.0000
	Ġdocument: 0.0479
	This: 0.1155
	.: 0.0479
	Ġfirst: 0.1155
	Ġis: 0.1155
Text 2:
	Ġthe: 0.0000
	Ġdocument: 0.0411
	This: 0.0990
	.: 0.0411
	Ġsecond: 0.3961
	Ġis: 0.0990
Text 3:
	Ġthe: 0.0000
	Ġone: 0.2773
	Ġthird: 0.2773
	And: 0.2773
	.: 0.0575
Text 4:
	Ġthis: 0.2310
	Ġthe: 0.0000
	Ġdocument: 0.0479
	?: 0.2310
	Ġfirst: 0.1155
	Is: 0.2310
